In [24]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
from transformers import BertTokenizer
import torch

In [4]:
# data_path = os.environ['DATA_PATH']
data_path = "D:/Downloads/ted_dataset.tar/ted_dataset/"
os.listdir(data_path)

['.DS_Store',
 'COPYRIGHT',
 'data.pkl',
 'encoded.pkl',
 'fix_metadata.py~',
 'README',
 'stats.py',
 'stats.py~',
 'targets.pkl',
 'ted_talks-10-Sep-2012.json',
 'ted_talks-25-Apr-2012.json',
 'ted_users-10-Sep-2012.json',
 'ted_users-25-Apr-2012.json',
 'test_data.pkl',
 'text.pkl',
 'texts.pkl',
 'train_data.pkl',
 'valid_data.pkl']

In [4]:
file_path = data_path + "ted_talks-25-Apr-2012.json"

In [4]:
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
transcripts = [d['transcript'] for d in data]

In [6]:
text = '\n'.join(transcripts)
len(text.split(' '))

2477012

In [7]:
escape_words = (' (Laughter) ', ' (Applause) ')

In [8]:
for ew in escape_words:
    text = text.replace(ew, '')

In [9]:
text = text.replace('!', '.')
text = text.replace(':', ',')
text = text.replace('--', ',')
text = text.replace('-', ',')
# text = text.replace('\"', '')

In [10]:
print(text[:10000])

Shall I ask for a show of hands or a clapping of people in different generations? I'm interested in how many are three to 12 years old.None, huh? All right. I'm going to talk about dinosaurs. Do you remember dinosaurs when you were that age?Dinosaurs are kind of funny, you know.We're going to kind of go in a different direction right now. I hope you all realize that. So I'll just give you my message up front, Try not to go extinct.That's it.People ask me a lot , in fact, one of the most asked questions I get is, why do children like dinosaurs so much? What's the fascination? And I usually just say, "Well dinosaurs were big, different and gone." They're all gone. Well that's not true, but we'll get to the goose in a minute. So that's sort of the theme, big, different and gone. The title of my talk, Shape,shifting Dinosaurs, The Cause of a Premature Extinction. Now I assume that we remember dinosaurs. And there's lots of different shapes. Lots of different kinds. A long time ago, back in

In [6]:
albert_pretrained = torch.hub.load('huggingface/pytorch-transformers', 'model', 'albert-base-v1')
albert_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'albert-base-v1')
albert_pretrained.eval()

Using cache found in C:\Users\gbenc/.cache\torch\hub\huggingface_pytorch-transformers_master
Using cache found in C:\Users\gbenc/.cache\torch\hub\huggingface_pytorch-transformers_master


AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0.1, inplace=False)
  

In [13]:
%%timeit
albert_tokenizer.tokenize(" , ! ? . ... .. asd. qwe.. tyu, uio , asdfs")

74.3 µs ± 444 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
%%timeit
albert_tokenizer.encode(" , ! ? . ... .. asd. qwe.. tyu, uio , asdfs")

188 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
target_ids = (9, 60, 15)
albert_tokenizer.encode(".?,")

[2, 13, 9, 60, 15, 3]

In [8]:
with open(data_path + 'encoded.pkl', 'rb') as f:
    encoded_texts = pickle.load(f)

In [9]:
albert_tokenizer.decode(encoded_texts[0][:100])

"[CLS] shall i ask for a show of hands or a clapping of people in different generations? i'm interested in how many are three to 12 years old.none, huh? all right. i'm going to talk about dinosaurs. do you remember dinosaurs when you were that age?dinosaurs are kind of funny, you know.we're going to kind of go in a different direction right now. i hope you all realize that. so i'll just give you my message"

In [11]:
id2target = {-1: 0,
              9: 1, # .
              60: 2, # ?
              15: 3  # ,
             }
target2id = {value: key for key, value in id2target.items()}
    
    
def create_target(encoded):
    targets = []
    text = []

    target = -1

    text.append(encoded[0])
    for word in encoded[1:]:#encoded_texts[0]:
        if word in target_ids:
            target = word
        else:
            targets.append(target)
            target = -1
            text.append(word)

    targets.append(target)

    targets = [id2target[t] for t in targets]

    return text, targets

In [35]:
texts = []
targets = []
for text in encoded_texts:
    text, target = create_target(text)
    texts.append(text)
    targets.append(target)

In [44]:
n = int(.8*len(texts))
test_n = int(.98*len(texts))
train_texts = texts[:n]
train_targets = targets[:n]
valid_texts = texts[n:test_n]
valid_targets = targets[n:test_n]
test_texts = texts[test_n:]
test_targets = targets[test_n:]
len(train_texts), len(valid_texts), len(test_texts)

(919, 207, 23)

In [7]:
# with open(data_path + 'train_data.pkl', 'wb') as f:
#     pickle.dump((train_texts, train_targets), f)
# with open(data_path + 'valid_data.pkl', 'wb') as f:
#     pickle.dump((valid_texts, valid_targets), f)
# with open(data_path + 'test_data.pkl', 'wb') as f:
#     pickle.dump((test_texts, test_targets), f)

with open(data_path + 'train_data.pkl', 'rb') as f:
    train_texts, train_targets = pickle.load(f)

In [8]:
# with open(data_path + "tokenized.pkl", 'wb') as f:
#     pickle.dump(tokenized_text, f)

In [9]:
from collections import Counter
c = Counter((t for targets in train_targets for t in targets))

In [ ]:
def get_weights_for_sampler(train_data):
    # used because of the imbalanced dataset
    train_data['sector_int'], _ = pd.factorize(train_data['sector'])

    class_sample_count = np.array(
        [len(np.where(train_data['sector_int'] == t)[0]) for t in np.unique(train_data['sector_int'])])
    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in train_data['sector_int']])

    samples_weight = torch.from_numpy(samples_weight)
    samples_weig

In [19]:
targets = np.array([t for targets in train_targets for t in targets])
targets.shape, targets[:2]

((2300323,), array([0, 0]))

In [23]:
output_dim = 4

weights = np.zeros((output_dim, ))
for t in range(output_dim):
    count = (targets == t).sum()
    weights[t] = count

weights /= weights.sum()
weights

array([0.86378348, 0.0553644 , 0.00459675, 0.07625538])

In [15]:
# .?,
s = 0
weights = []
for key, value in c.items():
    s += value
for key, value in c.items():
    print(key, target2id[key], albert_tokenizer.decode(target2id[key]) if key > 0 else "\"\"", value, value/s)
    weights.append(s/value)

0 -1 "" 1986981 0.8637834773638311
2 60 ? 10574 0.004596745761356122
1 9 . 127356 0.055364398825730125
3 15 , 175412 0.07625537804908267


In [18]:
import numpy as np
weights = np.array(weights)
weights /= weights.sum()
weights

array([0.00463303, 0.87060261, 0.07228361, 0.05248074])

In [26]:
sum(p.numel() for p in albert_pretrained.parameters())

11683584